In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")

os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
import bs4
from langchain_classic import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI , OpenAIEmbeddings

# Load Documents

loader = WebBaseLoader(
    web_path=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    ),
)

docs = loader.load()


# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# MMR 알고리즘 (Maximal Magginal Relevance)
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# RAG-Fusion: 관련 검색 쿼리 생성

# template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
# Generate multiple search queries related to: {question} \n
# Output (4 queries):"""


template = """당신은 주어진 하나의 질문을 기반으로 여러 검색 쿼리를 생성하는 유용한 조수입니다. \n
다음 질문과 관련된 여러 검색 쿼리를 생성하세요: {question} \n
출력 (4개의 쿼리):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)


## RRF
- RRF 점수 = 1 / (  K + 순위)
- k 는 정해진 상수로, 일반적으로 60 으로 할당
- 순위가 낮을수록 높은 점수를 부여해서, 여러 순위를 종합한 총점을 매겨서 최종 순위를 조정


In [ ]:
from langchain_classic.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60, top_n=2):
    """
    여러개의 순위가 매겨진 문서 리스트를 받아, RRF 공식을 사용하여 문서의 최종 순위를 계산하는 함수.
    k 는 RRF 공식에서 사용되는 선택적 파라미터
    top_n  은 반환할 우선순위가 높은 문서의 개수
    """

    # 각 고유한 문서에 대한 점수를 저장할 딕셔너리를 초기화
    fused_score = {}

    # 순위가 매겨진 문서 리스트를 순회합니다.
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)

            if doc_str not in fused_score:
                fused_score[doc_str] = 0
            previous_score = fused_score[doc_str]

            fused_score[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_score.items(), key=lambda x:x[1],
        reverse=True)
    ]

    return reranked_results[:top_n]